In [309]:
import numpy as np  

In [315]:
def hypothesis(theta, X, logistic=False):
    if logistic:
        return 1/(1 + np.exp(np.matmul(X, theta)))
    else:
        return np.matmul(X, theta)
        

In [331]:
def gradient_descent_regression(X, Y, theta, learning_rate, max_iters, regularization):
        # Update theta using the grdient descent formula.
        # Remember this requires simultaneous updating.
        prediction = hypothesis(theta, X)
        
        m = X.shape[0]
        
        difference = prediction - Y       
        
        # Calculate the sum.
        sigma = np.dot((prediction - Y)[:,0], X).reshape((-1,1))
        
        theta_0 = theta[0:1, :] - ((learning_rate / m) * sigma[0:1, :])
        theta_rest = theta[1:, :] * \
                        (1 - (learning_rate * regularization / m)) - ((learning_rate / m) * sigma[1:,:])

                                                                                
        return np.insert(theta_rest, 0, theta_0, axis=0)
        

In [332]:
def gradient_descent_logistic():
    pass

In [336]:
def gradient_descent(X, Y, learning_rate=0.01, max_iters=10000, regularization=0.1, logistic=False):
    """Returns the optimum theta using gradient descent.
       Remember that you may need to normalize your data.
       We use regu"""
    
    # Number of training examples.
    m = X.shape[0]

    # When true we stop calculating the optimum value of theta.
    done = False
    
    # We declare convergence if J(theta) decreases by less than below.
    convergence = 10 ** -7
    
    # Assign theta to be a value of 0s initially.
    theta = np.array([0 for x in range(X.shape[1] + 1)], dtype='float')
    theta = np.reshape(theta, (-1,1))
    # There should be as many thetas as features.
    
    
    # Need to add ones to X
    ones = np.ones((1,m))
    
    # Append the columns of ones to x
    X = np.insert(X, 0, ones, axis=1)        
    
    for i in range(max_iters):
        
        # Output if the maximum number of iterations has been reached.
        if i == max_iters - 1:
            print("The maximum number of iterations has been reached!")
        
        # Need to copy theta
        prev_theta = theta
                              
        if not logistic:
            theta = gradient_descent_regression(X, Y, theta ,learning_rate, max_iters, regularization)
        
        #print("Theta: " + str(theta), "Prev Theta: " + str(prev_theta))
        
        if abs((theta - prev_theta).max(axis=0)) < convergence:
            print("Converged!")
            break            
        
    return theta
          
X = [[1], [4], [3], [9], [22], [133], [33], [92], [44], [54], [66], [22], [27]]
Y = [[2], [8], [6], [18], [44], [266], [66], [184], [88], [108], [132], [44], [54]]
Y = np.array(Y, dtype='float')


X = np.array(X, dtype='float')

gradient_descent(X, Y, learning_rate=0.0001)

Converged!


array([[0.02660458],
       [1.99963987]])

In [321]:
def feature_scaling(X):
    """Uses mean normalization to scale the input."""
    
    # Calculate the mean of each feature.
    X_mean = np.mean(X, axis=0).reshape((-1,1))
    X_std = np.std(X, axis=0).reshape((-1,1))

    # If the standard deviation is 0 then we encounter problems.
    if X_std.min(axis=0)[0] == 0:
        return None
    
    return (X - X_mean.T) / X_std.T 